In [1]:
import os
import numpy as np
import cv2
from skimage.transform import resize
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from matplotlib.colors import to_rgba
from functions_EDX import *
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from skimage import exposure
from scipy.ndimage import gaussian_filter, uniform_filter
from skimage.filters import rank
from skimage.morphology import disk
from skimage.morphology import black_tophat, disk, white_tophat
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.kernel_approximation import Nystroem
import matplotlib.cm as cm

import seaborn as sns
from PIL import Image
from scipy.ndimage import uniform_filter, gaussian_filter
from scipy.spatial import cKDTree
from scipy.stats import norm

c:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
#perform mini batch k-means by fitting on the smoothed spectra and predicting on the raw
X_partial  = np.load(r"C:\Users\P301644\Documents\Grountruth\kpca_1_transfomed_data0.npy")
# X_partial_orig= np.load('X_partial_orig3489.npy')

from sklearn.cluster import SpectralClustering, MiniBatchKMeans
mbkmeans = MiniBatchKMeans(n_clusters=5, batch_size=10000, random_state=1)
mbkmeans.fit(X_partial)
labels = mbkmeans.predict(X_partial)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


AttributeError: 'NoneType' object has no attribute 'split'

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming some pre-existing global variables like X_partial_orig, labels, etc.

# Dummy cluster data
cluster_hierarchy = {}

# Dropdown to select the cluster to sub-cluster
cluster_dropdown = widgets.Dropdown(
    options=[i for i in range(10)],
    description='Cluster:',
    disabled=False,
)

# Textbox to input the number of sub-clusters
subcluster_input = widgets.IntText(
    value=2,
    description='Sub-clusters:',
    disabled=False
)

# Button to perform sub-clustering
button = widgets.Button(
    description='Sub-cluster'
)

# Button to go back to the previous level
back_button = widgets.Button(
    description='Back',
    disabled=True  # Initially disabled because there's no previous level yet
)

# Radio buttons to select the sub-cluster for further sub-clustering
subcluster_radio = widgets.RadioButtons(
    options=[],
    description='Sub-cluster:',
    disabled=True
)

# Output widget to display images and interaction
output = widgets.Output()

# Track the current level of clustering to facilitate recursive sub-clustering
current_level = {}

def display_selected_cluster(change):
    """Display the image of the selected cluster."""
    cluster = cluster_dropdown.value
    with output:
        clear_output(wait=True)
        plt.figure()
        plt.imshow((labels == cluster).reshape(1, 1024, 1024)[0])
        plt.title(f'Cluster {cluster}')
        plt.show()

    # Reset level tracking and disable the back button
    current_level[cluster] = 0
    back_button.disabled = True
    subcluster_radio.options = []
    subcluster_radio.disabled = True

def on_button_click(b):
    """Perform sub-clustering and update the sub-cluster radio buttons."""
    global cluster_hierarchy, current_level
    
    cluster = cluster_dropdown.value
    sub_clusters = subcluster_input.value
    
    from sklearn.cluster import MiniBatchKMeans
    
    # Determine the current level for the selected cluster
    level = current_level.get(cluster, 0)
    
    # Perform sub-clustering
    mbkmeans = MiniBatchKMeans(n_clusters=sub_clusters, batch_size=2000, random_state=1)
    image_cluster = np.zeros(1 * 1024 * 1024)
    
    if level == 0:
        mask = (labels == cluster)
    else:
        mask = cluster_hierarchy[cluster][level - 1][selected_subcluster]
    
    image_cluster[mask] = mbkmeans.fit_predict(X_partial[mask])
    
    # Store the sub-cluster information in the hierarchy
    if cluster not in cluster_hierarchy:
        cluster_hierarchy[cluster] = {}
    cluster_hierarchy[cluster][level] = {}
    
    for i in range(sub_clusters):
        cluster_hierarchy[cluster][level][i] = mask & (image_cluster == i)
    
    # Update radio button options for sub-clustering
    subcluster_radio.options = [f'Sub-cluster {i}' for i in range(sub_clusters)]
    subcluster_radio.disabled = False
    
    # Update the current level and enable the back button
    current_level[cluster] = level + 1
    back_button.disabled = False
    
    # Display the sub-cluster images
    with output:
        clear_output(wait=True)
        for i in range(sub_clusters):
            plt.figure()
            plt.imshow(cluster_hierarchy[cluster][level][i].reshape(1, 1024, 1024)[0])
            plt.title(f'Sub-cluster {i} of Cluster {cluster} (Level {level})')
            plt.show()

def on_subcluster_select(change):
    """Display the image of the selected sub-cluster for further sub-clustering."""
    global selected_subcluster
    
    selected_cluster = cluster_dropdown.value
    selected_subcluster = int(subcluster_radio.value.split()[-1])
    
    with output:
        clear_output(wait=True)
        level = current_level.get(selected_cluster, 0) - 1  # Previous level
        plt.figure()
        plt.imshow(cluster_hierarchy[selected_cluster][level][selected_subcluster].reshape(1, 1024, 1024)[0])
        plt.title(f'Sub-cluster {selected_subcluster} of Cluster {selected_cluster} (Level {level})')
        plt.show()

def on_back_button_click(b):
    """Go back to the previous level of sub-clustering."""
    selected_cluster = cluster_dropdown.value
    current_level[selected_cluster] -= 1
    
    if current_level[selected_cluster] == 0:
        back_button.disabled = True  # Disable back button if at the root level
    
    level = current_level[selected_cluster] - 1  # Previous level
    if level < 0:
        return  # No level to go back to
    
    # Update radio button options for the previous level
    subcluster_radio.options = [f'Sub-cluster {i}' for i in cluster_hierarchy[selected_cluster][level].keys()]
    subcluster_radio.disabled = False
    
    # Display the sub-cluster images for the previous level
    with output:
        clear_output(wait=True)
        for i in cluster_hierarchy[selected_cluster][level].keys():
            plt.figure()
            plt.imshow(cluster_hierarchy[selected_cluster][level][i].reshape(4, 1024, 1024)[0])
            plt.title(f'Sub-cluster {i} of Cluster {selected_cluster} (Level {level})')
            plt.show()

# Attach handlers
cluster_dropdown.observe(display_selected_cluster, names='value')
button.on_click(on_button_click)
back_button.on_click(on_back_button_click)
subcluster_radio.observe(on_subcluster_select, names='value')

# Display the widgets
display(cluster_dropdown, subcluster_input, button, back_button, subcluster_radio, output)


Dropdown(description='Cluster:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

IntText(value=2, description='Sub-clusters:')

Button(description='Sub-cluster', style=ButtonStyle())

Button(description='Back', disabled=True, style=ButtonStyle())

RadioButtons(description='Sub-cluster:', disabled=True, options=(), value=None)

...

c:\ProgramData\Anaconda3\envs\mitochondria\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


c:\ProgramData\Anaconda3\envs\mitochondria\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


c:\ProgramData\Anaconda3\envs\mitochondria\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


c:\ProgramData\Anaconda3\envs\mitochondria\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


c:\ProgramData\Anaconda3\envs\mitochondria\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [5]:
!pip install pacmap

  Using cached annoy-1.17.3.tar.gz (647 kB)
  Created wheel for annoy: filename=annoy-1.17.3-cp38-cp38-win_amd64.whl size=52796 sha256=cf941c562420c01b6d15a2fd4fcf0f82fc7b1d0968f838d65a48a5d93462fb4c
  Stored in directory: c:\users\p301644\appdata\local\pip\cache\wheels\55\2b\62\d6b83c564671d3cea4eb393ffdfc3ed9b9f2482fc79e8b4c35
Successfully built annoy
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.1
    Uninstalling llvmlite-0.39.1:
      Successfully uninstalled llvmlite-0.39.1
  Attempting uninstall: numba
    Found existing installation: numba 0.56.4
    Uninstalling numba-0.56.4:
      Successfully uninstalled numba-0.56.4


ERROR: mlflow 2.1.1 has requirement Flask<3, but you'll have flask 3.0.2 which is incompatible.
ERROR: mlflow 2.1.1 has requirement packaging<23, but you'll have packaging 23.0 which is incompatible.
ERROR: mlflow 2.1.1 has requirement pandas<2, but you'll have pandas 2.0.3 which is incompatible.
